In [103]:
import json

# Read json file
json_file = open('IOA_COVIDKhoj_data_export.json')
json = json.load(json_file)

# Iterate through json file structure and append each row to a list
# (not the most elegant, but it gets the job done better than json_normalize)
leads = []
for state in json:
    for service in json[state]: 
        for phone_number in json[state][service]:
            lead = {
                'State': state,
                'Services': service,
                'Phone Number': phone_number,
            }
            # Concatenates all the remaining lowest level fields to the dict 
            lead.update(json[state][service][phone_number])
            leads.append(lead)

In [107]:
import pandas as pd
import numpy as np
import phonenumbers

# Construct a pandas dataframe (this is where the fun begins!)
df = pd.DataFrame(leads)

# Renaming columns
df.rename(columns={
    'desc': 'Description',
    'area': 'Location',
    'city': 'City',
    'verified': 'Verified Timestamp',
    'name': 'Name'
}, inplace=True)

# Merge details for extra services into "Description" column
desc_mergers = {
    'remdesivir': 'Medicine name: Remdesivir',
    'fabiflu': 'Medicine name: Fabiflu',
    'tocilizumab': 'Medicine name: Tocilizumab',
    'ambulance': 'Notes: Ambulance Services', 
    'oxygen_bed': 'Notes: Has Oxygen Beds', 
    'icu_bed': 'Notes: Has ICU Bed', 
    'icu_bed with ventilator': 'Has ICU Bed with Ventilator'
}

for key, value in desc_mergers.items():
    df['Description'] += np.where(df['Services'] == key, ' ; ' + value, '')

# Merge extra location columns into "Location" column
for loc_merger in ['link', 'pincode']:
    df['Location'] += np.where(
            (df[loc_merger].notnull()) & (df[loc_merger] != ''),
             ' ; ' + df[loc_merger], '')

# Merge phone number columns
df['Phone Number'] += np.where(df['phone'].notnull(), ' ; ' + df['phone'], '')

# Trim out any excess semicolons on the sides
for column in ['Description', 'Phone Number', 'Location']:
    df[column] = df[column].str.strip(' ; ')

# Properly format some columns
df['State'] = df['State'].str.title() 
df['City'] = df['City'].str.title()
df['Verified Timestamp'] = pd.to_datetime(df['Verified Timestamp'], unit='ms', errors='ignore')

# Replace values that were already merged, misspelled values, etc.
df.replace({
    'beds': 'Hospital Beds',
    'oxygen': 'Oxygen',
    'remdesivir': 'Medicine', # Added to description
    'food': 'Food', 
    'other medicines': 'Medicine',
    'plasma': 'Blood / Plasma',
    'ambulance': 'Other', # Added to description
    'oxygen_cylinders': 'Oxygen',
    'teleconsultation': 'Telehealth',
    'oxygen_bed': 'Hospital Beds', # Added to description
    'fabiflu': 'Medicine', # Added to description
    'icu_bed': 'Hospital Beds', # Added to description
    'icu_bed with ventilator': 'Hospital Beds', # Added to description
    'only_bed': 'Hospital Beds',
    'tocilizumab': 'Medicine', # Added to description
    'mumbai': 'Mumbai'
}, inplace=True)

# Merge "date" and "time" columns into one "Timestamp of Entry" column
df['Original Entry Timestamp'] = pd.to_datetime(df['date'] + ' ' + df['time'], errors='ignore')

# Parse "Phone Number" and "Description" columns with the phonenumbers package
def parse_phone_numbers(phone_string):
    if isinstance(phone_string, str):
        phone_numbers = []
        for match in phonenumbers.PhoneNumberMatcher(phone_string, region='IN'):
            num = phonenumbers.parse(match.raw_string, "IN")
            num = phonenumbers.format_number(num, phonenumbers.PhoneNumberFormat.E164)
            phone_numbers.append(num)
        return phone_numbers

df['Phone Numbers'] = df['Phone Number'] + ' ' + df['Description']
df['Phone Numbers'] = df['Phone Numbers'].apply(parse_phone_numbers)

# Remove "\n" and "undefined" strings from "Description" column
df['Description'] = df['Description'].str.replace('\n|undefined', ' ', regex=True)
df[df['Original Entry Timestamp'].notnull()]['Original Entry Timestamp']

# Drop columns that aren't relevant or ones we've already merged
df = df.drop(columns=['firstVerifiedAt', 'phone', 'link', 'pincode', 'date', 'time'])

# Reorder our remaining columns
df = df[[
    'Name',
    'Original Entry Timestamp',
    'Verified Timestamp',
    'Description',
    'Services', 
    'Phone Numbers',
    'State',
    'City',
    'Location',
]]

df
# df.to_csv('IOA_COVIDKhoj_clean.csv', index=False)

,Name,Original Entry Timestamp,Verified Timestamp,Description,Services,Phone Numbers,State,City,Location
0,UNKNOWN,NaN,2021-04-30 09:39:12.320,Guntur Covid fighter's team: not verified: 81...,Hospital Beds,"[+919848940304, +918143222456, +919397602553, ...",Andhra Pradesh,NaN,NaN
1,UNKNOWN,NaN,2021-05-09 11:33:18.910,"not verified : 8712912987, 9989153803, 9849122...",Oxygen,"[+918712912987, +918712912987, +919989153803, ...",Andhra Pradesh,NaN,NaN
2,Vinay,NaN,2021-05-01 13:41:13.368,possibly available on 02/05/21,Oxygen,[+919350068204],Andhra Pradesh,NaN,NaN
3,Rafi,NaN,2021-04-29 04:41:02.176,"chinna waltair, vizag, landmark: near reliance...",Oxygen,[+919966012368],Andhra Pradesh,NaN,NaN
4,Mylan helpline,NaN,2021-04-30 14:28:27.925,All India Helpline for Remdesvir ; Medicine ...,Medicine,[+917829980066],Andhra Pradesh,NaN,NaN
...,...,...,...,...,...,...,...,...,...
619,Dr. Nikita Rajkondawar,23 April 17:47,NaT,available through phone calls from 4 to 9 PM.s...,Telehealth,[+919552555677],Rajasthan,Fatehpur,
620,"GRS Life Care, Madurai",23 April 4:58:02 PM,NaT,they can arrange in 5-6 days ; Medicine name: ...,Medicine,[+919367526015],Telangana,Hyderabad,"1st Floor, 159 G,Thalai Veethi 2 Cross Street,..."
621,Keshav Mishra,NaN,2021-04-26 16:58:57.087,NaN,Hospital Beds,None,West Bengal,NaN,NaN
622,Prabal Chaudhury,NaN,2021-05-01 10:15:17.295,Refilling in Baghajatin area They'll test whet...,Oxygen,[+919830083350],West Bengal,NaN,NaN
